In [ ]:
from pyspark.sql import SQLContext
from pyspark import SparkContext

sc = SparkContext()
sqlContext = SQLContext(sc)

# Read data from ``data/hotels_bahamas_min.csv``

Read minimal data about hotels in the Bahamas from the CSV file an store them in temporary table ``hotels``.

In [ ]:
df = sqlContext.read.format('csv').options(header='true', inferschema='true').load('/home/lukas/spark/data/hotels_bahamas_min.csv')
df.registerTempTable("hotels")
sqlContext.sql("SELECT * FROM hotels").show()
sqlContext.sql("SELECT * FROM hotels SKYLINE OF price MIN, distance MIN").show()

In [ ]:
sqlContext.sql("SELECT * FROM hotels").explain("extended")
sqlContext.sql("SELECT * FROM hotels SKYLINE OF price MIN, distance MIN").explain()
sqlContext.sql("SELECT * FROM hotels SKYLINE OF price MIN, distance MIN").explain("extended")

In [ ]:
df.filter("price > 200").skyline(["price", "distance"], minMaxDiff=["min", "min"]).explain("extended")
df.filter("price > 200").skyline(["price", "distance"], minMaxDiff=["min", "min"], distinct=[1, 0]).explain("extended")

In [ ]:
# df.filter("price > 200").sort(df.price.desc()).explain("extended")
df.filter("price > 200").skyline(df.price.smin().sdistinct(), df.distance.smax().sdistinct()).explain("extended")